In [ ]:
%load_ext autotime

In [ ]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import numpy as np
import copy
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import stats
import seaborn as sns
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn import metrics
from gensim.models import Word2Vec

In [ ]:
df_annotations=  pd.read_csv('full_cleaned_dataset.csv')
df_annotations.head() 

In [ ]:
def TFIDF(data, max_feat, n_gram):
    vect=TfidfVectorizer(max_features=max_feat, stop_words= stopwords.words('dutch'), token_pattern=r'\b[^\d\W][^\d\W]+\b', ngram_range=n_gram).fit(data)
    X= vect.fit_transform(data)
    X=pd.DataFrame(X.toarray(), columns= vect.get_feature_names())
    return X

In [ ]:
def y_set(dataset_column):
    y= dataset_column
    y = y.replace('NEUTRAL','1')
    y = y.replace('POSITIVE','2')
    y = y.replace('NEGATIVE','0')
    for i in y:
        i=int(i)
    new=[]
    for i in y:
        new.append(int(i))
    
    return new

In [ ]:
def cf_mat(y_test, y_pred):
    mat=confusion_matrix(y_test, y_pred)/len(y_test)
    return sns.heatmap(mat, annot=True)

In [ ]:
def MNaiveBayes(X, y_inputkolom):
    y= y_set(y_inputkolom)
    
    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Build a logistic regression
    model = MultinomialNB()
    #model = ComplementNB()
    #model = GaussianNB()
    #model = BernoulliNB()
    mnb=model.fit(X_train, y_train)
    
    # Predict the labels 
    y_predict = mnb.predict(X_test)
    new=[]
    for i in y_test:
        new.append(int(i))
        
    y_pred=[]
    for i in y_predict:
        y_pred.append(int(i))
    print(' The Macro F1 measure is', metrics.f1_score(new, y_pred, average='macro'))
    print(' The Macro precision is',metrics.precision_score(new, y_pred, average='macro'))
    print(' The Macro recall is',metrics.recall_score(new, y_pred, average='macro'))
    
    print('The Accuracy of the model is', accuracy_score(y_test, y_predict))
    print('The Spearman correlation of the model is ', stats.spearmanr(y_test, y_predict))
    print('The Kendall tau of the model is ', stats.kendalltau(y_test,y_predict))
    print('The confusion matrix looks as follows', cf_mat(y_test, y_predict))
    #return y_predict, y_test, log_reg

In [ ]:
def BNaiveBayes(X, y_inputkolom, binary= 0.0):
    y= y_set(y_inputkolom)
    
    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Build a Naive Bayes model
    model = BernoulliNB(binarize= binary)
    mnb=model.fit(X_train, y_train)
    
    # Predict the labels 
    y_predict = mnb.predict(X_test)
    
    new=[]
    for i in y_test:
        new.append(int(i))
        
    y_pred=[]
    for i in y_predict:
        y_pred.append(int(i))
    print(' The Macro F1 measure is', metrics.f1_score(new, y_pred, average='macro'))
    print(' The Macro precision is',metrics.precision_score(new, y_pred, average='macro'))
    print(' The Macro recall is',metrics.recall_score(new, y_pred, average='macro'))
    
    print('The Accuracy of the model is', accuracy_score(y_test, y_predict))
    print('The Spearman correlation of the model is ', stats.spearmanr(y_test, y_predict))
    print('The Kendall tau of the model is ', stats.kendalltau(y_test,y_predict))
    print('The confusion matrix looks as follows', cf_mat(y_test, y_predict))
    #return y_predict, y_test, log_reg

In [ ]:
def MNaiveBayes_val_tuning(X,y_inputkolom):
    y= y_set(y_inputkolom)
    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
    

    # Build a logistic regression
    model = MultinomialNB()
    mnb=model.fit(X_train, y_train)
    
    # Predict the labels 
    y_predict = mnb.predict(X_val)
    acc_score= accuracy_score(y_val, y_predict)
    print('The Accuracy of the model is', acc_score)
    #print('The confusion matrix looks as follows', cf_mat(y_val, y_predict))
    return acc_score

In [ ]:
def BNaiveBayes_val_tuning(X,y_inputkolom, binary=0.0):
    y= y_set(y_inputkolom)
    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
    

    # Build a logistic regression
    model = BernoulliNB(binarize=binary)
    cnb=model.fit(X_train, y_train)
    
    # Predict the labels 
    y_predict = cnb.predict(X_val)
    acc_score= accuracy_score(y_val, y_predict)
    print('The Accuracy of the model is', acc_score)
    #print('The confusion matrix looks as follows', cf_mat(y_val, y_predict))
    return acc_score

# Multinomial Naive Bayes

## TFIDF

### Plain text

In [ ]:
plotje=[]
max_feat_list=[100,300,500,750 ,1000, 1100, 1500, 2000, 2500, 3000, 4000,5000 , 6000, 8000, 10000, 12000]
for i in max_feat_list:
    print('the number of max_features is', i)
    X= TFIDF(df_annotations.text.values.astype('U'),i,(1,1))
    acc_score=MNaiveBayes_val_tuning(X, df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_feat_list, plotje)
plt.show()

In [ ]:
for i in [(1,1),(1,2), (2,2), (1,3), (2,3), (3,3)]:
    X=TFIDF(df_annotations.text.values.astype('U'),8000,i)
    MNaiveBayes_val_tuning(X,df_annotations['label'])

In [ ]:
X= TFIDF(df_annotations.text.astype('U'), 8000,(1,1))
MNaiveBayes(X, df_annotations['label'])

### Stemmed

In [ ]:
plotje=[]
max_feat_list=[100,300,500,750 ,1000, 1100, 1500, 2000, 2500, 3000, 4000,5000 , 6000, 8000, 10000, 12000]
for i in max_feat_list:
    print('the number of max_features is', i)
    X= TFIDF(df_annotations.processed_annotations_joined.values.astype('U'),i,(1,1))
    acc_score=MNaiveBayes_val_tuning(X, df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_feat_list, plotje)
plt.show()

In [ ]:
for i in [(1,1),(1,2), (2,2), (1,3), (2,3), (3,3)]:
    X=TFIDF(df_annotations.processed_annotations_joined.values.astype('U'),10000,i)
    MNaiveBayes_val_tuning(X,df_annotations['label'])

In [ ]:
X= TFIDF(df_annotations.processed_annotations_joined.astype('U'), 10000,(1,1))
MNaiveBayes(X, df_annotations['label'])

### Lemmatized

In [ ]:
plotje=[]
max_feat_list=[100,300,500,750 ,1000, 1100, 1500, 2000, 2500, 3000, 4000,5000 , 6000, 8000, 10000, 12000, 15000]
for i in max_feat_list:
    print('the number of max_features is', i)
    X= TFIDF(df_annotations.processed_annotations_lemmatized_joined.values.astype('U'),i,(1,1))
    acc_score=MNaiveBayes_val_tuning(X, df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_feat_list, plotje)
plt.show()

In [ ]:
for i in [(1,1),(1,2), (2,2), (1,3), (2,3), (3,3)]:
    X=TFIDF(df_annotations.processed_annotations_lemmatized_joined.values.astype('U'),12000,i)
    MNaiveBayes_val_tuning(X,df_annotations['label'])

In [ ]:
X= TFIDF(df_annotations.processed_annotations_lemmatized_joined.astype('U'), 12000,(1,2))
MNaiveBayes(X, df_annotations['label'])

## Word2Vec

In [ ]:
def sent(data_column):
    sentences=[]
    for i in range(len(df_annotations['text'])):
        words_in_sentence=df_annotations['text'][i].split()
        sentences.append(words_in_sentence)
    return sentences

In [ ]:
def W2V(sent, vector_s, wind=5, min_c=1, work=4, sg=1, alpha= 0.025):
    model= Word2Vec(sentences=sent, vector_size=vector_s, window=wind, min_count=min_c, workers=work, sg=sg, alpha=alpha)
    Word2Vec_text_av=[]
    for i in range(len(df_annotations['text'])):
        l=[]
        for word in df_annotations['text'][i].split():
            if word in model.wv.key_to_index:
                l.append(list(model.wv.get_vector(word)))
            else:
                l.append([0]*vector_s)
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        Word2Vec_text_av.append(avg)
    X=pd.DataFrame(Word2Vec_text_av)
    return X

In [ ]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

### Plain text

In [ ]:
plotje=[]
max_vect= [50,100,200,300,500,600,700,1000,1200,1500,1800,2000,2500,3000]
for i in max_vect:
    X=W2V(sent(df_annotations['text']), i)
    acc_score=MNaiveBayes_val_tuning(normalize(X), df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_vect, plotje)
plt.show()

In [ ]:
plotje=[]
wind= [1,2,3,4,5,6,7,8,9,10, 12, 15, 20]
for i in wind:
    X=W2V(sent(df_annotations['text']), 600, i)
    acc_score=MNaiveBayes_val_tuning(normalize(X), df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('window')
plt.ylabel('Accuracy')
plt.plot(wind, plotje)
plt.show()

In [ ]:
plotje=[]
wind= [ 20, 25, 30,35,40,45,50]
for i in wind:
    X=W2V(sent(df_annotations['text']), 600, i)
    acc_score=MNaiveBayes_val_tuning(normalize(X), df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('window')
plt.ylabel('Accuracy')
plt.plot(wind, plotje)
plt.show()

In [ ]:
plotje=[]
min_c= [0,1,2,3,4,5,6,7,8,9,10,20,30,50]
for i in min_c:
    X=W2V(sent(df_annotations['text']), 600, 40,i)
    acc_score=MNaiveBayes_val_tuning(normalize(X), df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('min_count')
plt.ylabel('Accuracy')
plt.plot(min_c, plotje)
plt.show()

In [ ]:
plotje=[]
for i in [0,1]:
    X=W2V(sent(df_annotations['text']), 600, 40,3,16,i)
    acc_score=MNaiveBayes_val_tuning(normalize(X), df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
plotje=[]
alphas= [0.01,0.025,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.8,1]
for i in alphas:
    X=W2V(sent(df_annotations['text']), 600, 40,3,16,1,i)
    acc_score=MNaiveBayes_val_tuning(normalize(X), df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('Alpha')
plt.ylabel('Accuracy')
plt.plot(alphas, plotje)
plt.show()

In [ ]:
X=W2V(sent(df_annotations['text']), 600, 40,4,16,1,0.05)
MNaiveBayes(normalize(X), df_annotations['label'])

### Lemmatized column

In [ ]:
def W2V_lem(sent, vector_s, wind=5, min_c=1, work=4, sg=1, alpha= 0.025):
    model= Word2Vec(sentences=sent, vector_size=vector_s, window=wind, min_count=min_c, workers=work, sg=sg, alpha=alpha)
    Word2Vec_text_av=[]
    for i in range(len(df_annotations['processed_annotations_lemmatized_joined'])):
        l=[]
        if type(df_annotations['processed_annotations_lemmatized_joined'][i])== float:
            l.append([0]*vector_s)
        else:
            for word in df_annotations['processed_annotations_lemmatized_joined'][i].split():
                if word in model.wv.key_to_index:
                    l.append(list(model.wv.get_vector(word)))
                else:
                    l.append([0]*vector_s)
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        Word2Vec_text_av.append(avg)
    X=pd.DataFrame(Word2Vec_text_av)
    return X

In [ ]:
plotje=[]
max_vect= [50,100,200,300,500,600,700,1000]
for i in max_vect:
    X=W2V_lem(sent(df_annotations['processed_annotations_lemmatized_joined']), i)
    acc_score=MNaiveBayes_val_tuning(normalize(X), df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_vect, plotje)
plt.show()

In [ ]:
plotje=[]
wind= [1,2,3,4,5,6,7,8,9,10, 12, 15, 20,25,30,35,40,45,50]
for i in wind:
    X=W2V_lem(sent(df_annotations['processed_annotations_lemmatized_joined']), 600, i)
    acc_score=MNaiveBayes_val_tuning(normalize(X), df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('window')
plt.ylabel('Accuracy')
plt.plot(wind, plotje)
plt.show()

In [ ]:
plotje=[]
min_c= [0,1,2,3,4,5,6,7,8,9,10,20,30,50]
for i in min_c:
    X=W2V_lem(sent(df_annotations['processed_annotations_lemmatized_joined']), 600, 35,i)
    acc_score=MNaiveBayes_val_tuning(normalize(X), df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('min_count')
plt.ylabel('Accuracy')
plt.plot(min_c, plotje)
plt.show()

In [ ]:
plotje=[]
for i in [0,1]:
    X=W2V_lem(sent(df_annotations['processed_annotations_lemmatized_joined']), 600, 35,2,16,i)
    acc_score=MNaiveBayes_val_tuning(normalize(X), df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
plotje=[]
alphas= [0.01,0.025,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.8,1]
for i in alphas:
    X=W2V_lem(sent(df_annotations['processed_annotations_lemmatized_joined']), 600, 35,2,16,1,i)
    acc_score=MNaiveBayes_val_tuning(normalize(X), df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('Alpha')
plt.ylabel('Accuracy')
plt.plot(alphas, plotje)
plt.show()

In [ ]:
X=W2V_lem(sent(df_annotations['processed_annotations_lemmatized_joined']), 600, 35,2,16,1,0.025)
MNaiveBayes(normalize(X), df_annotations['label'])

## Word2Vec pretrained

In [ ]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)

In [ ]:
def W2V_pretrained_emb(data):
    Word2Vec_text_av=[]
    for i in range(len(data)):
        l=[]
        
        if type(data[i])== float:
            l.append([0]*300)
        else:
            for word in data[i].split():
                if word in model.key_to_index:
                    l.append(list(model.get_vector(word)))
                else:
                    l.append([0]*300)
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        Word2Vec_text_av.append(avg)
    X=pd.DataFrame(Word2Vec_text_av)
    return X

In [ ]:
X=W2V_pretrained_emb(df_annotations['text'])
MNaiveBayes(normalize(X), df_annotations['label'])

In [ ]:
X=W2V_pretrained_emb(df_annotations['processed_annotations_lemmatized_joined'])
MNaiveBayes(normalize(X), df_annotations['label'])

## Pretrained Fasttext

In [ ]:
import fasttext
import fasttext.util

In [ ]:
model = fasttext.load_model('cc.nl.300.bin')

In [ ]:
def fasttext_pretrained_emb(data):
    fasttext_text_av=[]
    for i in range(len(data)):
        l=[]
        
        if type(data[i])== float:
            l.append([0]*300)
        else:
            for word in data[i].split():
                l.append(list(model.get_word_vector(word)))
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        fasttext_text_av.append(avg)
    X=pd.DataFrame(fasttext_text_av)
    return X

In [ ]:
X=fasttext_pretrained_emb(df_annotations['text'])
MNaiveBayes(normalize(X), df_annotations['label'])

In [ ]:
X=fasttext_pretrained_emb(df_annotations['processed_annotations_lemmatized_joined'])
MNaiveBayes(normalize(X), df_annotations['label'])

# Bernoulli Naive Bayes 

## TFIDF

### Plain text

In [ ]:
plotje=[]
max_feat_list=[100,300,500,750 ,1000, 1100, 1500, 2000, 2500, 3000, 4000,5000 , 6000, 8000, 10000, 12000,15000]
for i in max_feat_list:
    print('the number of max_features is', i)
    X= TFIDF(df_annotations.text.values.astype('U'),i,(1,1))
    acc_score=BNaiveBayes_val_tuning(X, df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_feat_list, plotje)
plt.show()

In [ ]:
for i in [(1,1),(1,2), (2,2), (1,3), (2,3), (3,3)]:
    X=TFIDF(df_annotations.text.values.astype('U'),8000,i)
    BNaiveBayes_val_tuning(X,df_annotations['label'])

In [ ]:
X= TFIDF(df_annotations.text.astype('U'), 8000,(1,1))
BNaiveBayes(X, df_annotations['label'])

### Stemmed

In [ ]:
plotje=[]
max_feat_list=[100,300,500,750 ,1000, 1100, 1500, 2000, 2500, 3000, 4000,5000 , 6000, 8000, 10000, 12000,13000,14000,15000]
for i in max_feat_list:
    print('the number of max_features is', i)
    X= TFIDF(df_annotations.processed_annotations_joined.values.astype('U'),i,(1,1))
    acc_score=BNaiveBayes_val_tuning(X, df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_feat_list, plotje)
plt.show()

In [ ]:
for i in [(1,1),(1,2), (2,2), (1,3), (2,3), (3,3)]:
    X=TFIDF(df_annotations.processed_annotations_joined.values.astype('U'),12000,i)
    BNaiveBayes_val_tuning(X,df_annotations['label'])

In [ ]:
X= TFIDF(df_annotations.processed_annotations_joined.astype('U'), 12000,(1,1))
BNaiveBayes(X, df_annotations['label'])

### Lemmatized

In [ ]:
plotje=[]
max_feat_list=[100,300,500,750 ,1000, 1100, 1500, 2000, 2500, 3000, 4000,5000 , 6000, 8000, 10000, 12000,13000,14000,15000,16000]
for i in max_feat_list:
    print('the number of max_features is', i)
    X= TFIDF(df_annotations.processed_annotations_lemmatized_joined.values.astype('U'),i,(1,1))
    acc_score=BNaiveBayes_val_tuning(X, df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_feat_list, plotje)
plt.show()

In [ ]:
for i in [(1,1),(1,2), (2,2), (1,3), (2,3), (3,3)]:
    X=TFIDF(df_annotations.processed_annotations_lemmatized_joined.values.astype('U'),12000,i)
    BNaiveBayes_val_tuning(X,df_annotations['label'])

In [ ]:
X= TFIDF(df_annotations.processed_annotations_lemmatized_joined.astype('U'), 12000,(1,2))
BNaiveBayes(X, df_annotations['label'])

## Word2Vec

### Plain text

In [ ]:
plotje=[]
max_vect= [50,100,200,300,500,600,700,1000]
for i in max_vect:
    X=W2V(sent(df_annotations['text']), i)
    acc_score=BNaiveBayes_val_tuning(X, df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_vect, plotje)
plt.show()

In [ ]:
plotje=[]
wind= [1,2,3,4,5,6,7,8,9,10, 12, 15, 20,25,30,35,40,45,50]
for i in wind:
    X=W2V(sent(df_annotations['text']), 500, i)
    acc_score=BNaiveBayes_val_tuning(X, df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
plotje=[]
wind= [50,55,60,65,70,75,80,85,90,100]
for i in wind:
    X=W2V(sent(df_annotations['text']), 500, i)
    acc_score=BNaiveBayes_val_tuning(X, df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('window')
plt.ylabel('Accuracy')
plt.plot(wind, plotje)
plt.show()

In [ ]:
plotje=[]
min_c= [0,1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100]
for i in min_c:
    X=W2V(sent(df_annotations['text']), 500, 65,i)
    acc_score=BNaiveBayes_val_tuning(X, df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('min_count')
plt.ylabel('Accuracy')
plt.plot(min_c, plotje)
plt.show()

In [ ]:
plotje=[]
for i in [0,1]:
    X=W2V(sent(df_annotations['text']), 500, 65,6,16,i)
    acc_score=BNaiveBayes_val_tuning(X, df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
plotje=[]
alphas= [0.01,0.025,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.8,1]
for i in alphas:
    X=W2V(sent(df_annotations['text']), 500, 56,6,16,1,i)
    acc_score=BNaiveBayes_val_tuning(X, df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('Alpha')
plt.ylabel('Accuracy')
plt.plot(alphas, plotje)
plt.show()

In [ ]:
X=W2V(sent(df_annotations['text']), 500, 56,6,16,1,0.05)
BNaiveBayes(X, df_annotations['label'])

### Lemmatized column

In [ ]:
def W2V_lem(sent, vector_s, wind=5, min_c=1, work=4, sg=1, alpha= 0.025):
    model= Word2Vec(sentences=sent, vector_size=vector_s, window=wind, min_count=min_c, workers=work, sg=sg, alpha=alpha)
    Word2Vec_text_av=[]
    for i in range(len(df_annotations['processed_annotations_lemmatized_joined'])):
        l=[]
        if type(df_annotations['processed_annotations_lemmatized_joined'][i])== float:
            l.append([0]*vector_s)
        else:
            for word in df_annotations['processed_annotations_lemmatized_joined'][i].split():
                if word in model.wv.key_to_index:
                    l.append(list(model.wv.get_vector(word)))
                else:
                    l.append([0]*vector_s)
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        Word2Vec_text_av.append(avg)
    X=pd.DataFrame(Word2Vec_text_av)
    return X

In [ ]:
plotje=[]
max_vect= [50,100,200,300,500,600,700,1000]
for i in max_vect:
    X=W2V_lem(sent(df_annotations['processed_annotations_lemmatized_joined']), i)
    acc_score=BNaiveBayes_val_tuning(X, df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_vect, plotje)
plt.show()

In [ ]:
plotje=[]
wind= [1,2,3,4,5,6,7,8,9,10, 12, 15, 20]
for i in wind:
    X=W2V_lem(sent(df_annotations['processed_annotations_lemmatized_joined']), 600, i)
    acc_score=BNaiveBayes_val_tuning(X, df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('window')
plt.ylabel('Accuracy')
plt.plot(wind, plotje)
plt.show()

In [ ]:
plotje=[]
min_c= [0,1,2,3,4,5,6,7,8,9,10,20,30,50]
for i in min_c:
    X=W2V_lem(sent(df_annotations['processed_annotations_lemmatized_joined']), 600, 15,i)
    acc_score=BNaiveBayes_val_tuning(X, df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('min_count')
plt.ylabel('Accuracy')
plt.plot(min_c, plotje)
plt.show()

In [ ]:
plotje=[]
for i in [0,1]:
    X=W2V_lem(sent(df_annotations['processed_annotations_lemmatized_joined']), 600, 15,7,16,i)
    acc_score=BNaiveBayes_val_tuning(X, df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
plotje=[]
alphas= [0.01,0.025,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.8,1]
for i in alphas:
    X=W2V_lem(sent(df_annotations['processed_annotations_lemmatized_joined']), 600, 15,7,16,1,i)
    acc_score=BNaiveBayes_val_tuning(X, df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('Alpha')
plt.ylabel('Accuracy')
plt.plot(alphas, plotje)
plt.show()

In [ ]:
X=W2V_lem(sent(df_annotations['processed_annotations_lemmatized_joined']), 600, 15,7,16,1,0.1)
BNaiveBayes(X, df_annotations['label'])

## Word2Vec pretrained

In [ ]:
X=W2V_pretrained_emb(df_annotations['text'])
BNaiveBayes(X, df_annotations['label'])

In [ ]:
X=W2V_pretrained_emb(df_annotations['processed_annotations_lemmatized_joined'])
BNaiveBayes(X, df_annotations['label'])

## Pretrained Fasttext

In [ ]:
X=fasttext_pretrained_emb(df_annotations['text'])
BNaiveBayes(X, df_annotations['label'])

In [ ]:
X=fasttext_pretrained_emb(df_annotations['processed_annotations_lemmatized_joined'])
BNaiveBayes(X, df_annotations['label'])